# Data Breach Transformation using Pandas

This notebook is to setup/start a local postgres sql server and transform the data breach data and load it to the postgres database.

## Install and start Postgresql (Only for mac)

In [ ]:
import subprocess

# Install PostgreSQL with Homebrew
install_postgres_command = "brew install postgresql@15"
subprocess.run(install_postgres_command, shell=True)

# Start PostgreSQL service
start_postgres_command = "brew services start postgresql"
subprocess.run(start_postgres_command, shell=True)

# Create postgres user
user_create_command = "/opt/homebrew/bin/createuser -s postgres"
subprocess.run(user_create_command, shell=True)

## Extract (from CSV), Transform (Using Python) and Load (Insert data to postgres)

The transform is implemented using the `Pandas` and sql connections are using `sqlalchemy`.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

Read the data from the CSV and load it into a Pandas DataFrame. The data has two columns Serial a duplicate column with the serial numbers and Sources which is not relevant for the use case. So we are dropping those two columns.

In [3]:
df = pd.read_csv('data_breaches.csv')
df = df.drop(columns=['Serial', 'Sources'])

Once data is in the DataFrame, now the `Records` column in the data has both numerics and strings. Whereas the strings are arbitrary unknown values. So to cleanse and load the data into postgres, we will check if the rows in `records` are numeric if not make it a `NaN` value.

Once all the strings are in `NaN` type, we then cast the column to an `int` type and fill the `NaN` as `0`

In [ ]:
df['Records'] = pd.to_numeric(df['Records'], errors='coerce')
df = df.fillna(0).astype({'Records': 'int'})

Let move on to the `Year` column, it has some inconsistencies in the some rows where as two years split with either `-` or `and` ex: `2014-2015` or `2014 and 2015`. Here we can use the [`iterrow`](https://pandas.pydata.org/docs/user_guide/basics.html#iteration) to split the insconsistent year rows into two but it is not suggested by Pandas does not suggest using it. So we did a `split` on the whole column based on the string as the column is already in string format and `explode`` it back to the DataFrame with re-indexing.

In [4]:
# Split rows with "-" string
df['Year'] = df['Year'].str.split('-')
df = df.explode('Year', ignore_index=True)

# Split the rows with " and " into two rows
df['Year'] = df['Year'].str.split(' and ')
df = df.explode('Year', ignore_index=True)

Now if we check the columns types both `Records` and `Year` should be an `int` type.

In [ ]:
df.dtypes

Load the data into the postgresql server using `sqlalchemy` package and create a engine. We used `if_exists='replace'` so that data will be replaced instead of failing while loading it and not load the index which is set to `False`

In [8]:
db_url = "postgresql://postgres@localhost/breach"
engine = create_engine(db_url)
df.to_sql('cleansed', engine, if_exists='replace', index=False)

355